## Importing required modules

In [65]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
import base64
from bs4 import BeautifulSoup as bs
import re
import pandas as pd

In [8]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
with open('token.json', 'rb') as token:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('gmail', 'v1', credentials=creds)
profile = service.users().getProfile(userId='me').execute()
profile

{'emailAddress': 'dongoodygoody@gmail.com',
 'messagesTotal': 3813,
 'threadsTotal': 3081,
 'historyId': '2849340'}

In [9]:
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
with open('token.json', 'rb') as token:
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)

service = build('gmail', 'v1', credentials=creds)

In [10]:
filter = service.users().messages().list(userId = 'me', maxResults=10,
                                         q = 'from:no_reply@accessbankplc.com \
                                             subject:AccessAlert Transaction Alert').execute()

filter_id = filter['messages']
id_lst = []
for ids in filter_id:
    id_lst.append(ids['id'])
    

In [15]:
snippet_lst = []
datetime = []
for each_id in id_lst:
    msgs = service.users().messages().get(userId = 'me', id = each_id).execute()
    snippet = msgs['snippet']
    date = msgs['payload']['headers'][-1]
    snippet_lst.append(snippet)
    datetime.append(date)

In [78]:
trans_lst = []
for each_id in id_lst:
    msgs = service.users().messages().get(userId = 'me', id = each_id).execute()
    snippet = msgs['snippet']
    date = msgs['payload']['headers'][-1]['value']
    trans_lst.append({
        'trans': snippet,
        'datetime': date
    })

In [92]:
trans_lst = []
for each_id in id_lst:
    msgs = service.users().messages().get(userId = 'me', id = each_id).execute()
    snippet = msgs['snippet']
    date = msgs['payload']['headers'][-1]['value']
    amount = re.search('\d*\.+\d+', snippet).group()
    acct_no = re.search('\d*\*+\d+', snippet).group()
    trans_lst.append({
        'amount': amount,
        'acct': acct_no,
        'datetime': date
    })
cols_name = ['amount', 'acct', 'datetime']
data = pd.DataFrame(trans_lst, columns=cols_name)

In [138]:
payload = msgs['payload']
headers = payload['headers']

parts = payload.get('body')['data']
parts = parts.replace('-','+').replace('_','/')
decode = base64.b64decode(parts)
soup = bs(decode,"lxml" )
body = soup.body()
body

[<center style="width: 100%; margin: 0; padding: 0; background-color: #f3f3f3;">
 <!--[if mso | IE]>
     <table role="presentation" border="0" cellpadding="0" cellspacing="0" width="100%" style="background-color: #f3f3f3;">
     <tr>
     <td>
     <![endif]-->
 <!--
             Set the email width. Defined in two places:
             1. max-width for all clients except Desktop Windows Outlook, allowing the email to squish on narrow but never go wider than 680px.
             2. MSO tags for Desktop Windows Outlook enforce a 680px width.
             Note: The Fluid and Responsive templates have a different width (600px). The hybrid grid is more "fragile", and I've found that 680px is a good width. Change with caution.
         -->
 <div class="email-container" style="max-width: 680px; margin: 0 auto;">
 <!--[if mso]>
             <table align="center" role="presentation" cellspacing="0" cellpadding="0" border="0" width="680">
             <tr>
             <td>
             <![endif

In [140]:
d = soup.find_all('tr')[7]
e = d.find_all('td')[6].text.replace("\n", ' ')
d

<tr>
<td style="padding: 20px 20px 0px; background: #ffffff; font-family: 'Montserrat', '-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Helvetica', 'Arial', 'sans-serif', 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol'; font-size: 15px; line-height: 20px; color: #616874;">
<h2 style="margin: 0 0 10px 0; font-family: 'Montserrat', '-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Helvetica', 'Arial', 'sans-serif', 'Apple Color Emoji', 'Segoe UI Emoji', 'Segoe UI Symbol'; font-size: 16px; line-height: 22px; color: #616874; font-weight: 600;">Transaction Summary</h2>
<div style="max-width: 680px; margin: 0 auto; overflow: auto;">
<!--[if mso]>
                            <table align="center" role="presentation" cellspacing="0" cellpadding="0" border="0" width="680">
                            <tr>
                            <td>
                            <![endif]-->
<table border="0" cellpadding="0" cellspacing="0" role="presentation" style=" width

In [141]:
headers

[{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
 {'name': 'Received',
  'value': 'by 2002:a05:6f02:e:b0:23:4722:82d with SMTP id 14csp17351rcj;        Thu, 1 Sep 2022 02:47:43 -0700 (PDT)'},
 {'name': 'X-Google-Smtp-Source',
  'value': 'AA6agR6KN936Sxruhv1Vo4OJBM0cuZo6Cq4O8Pgb2FDlXQJqvLXtwHs9lqaTJV/APtB24fWwKd8u'},
 {'name': 'X-Received',
  'value': 'by 2002:a05:620a:2601:b0:6bc:70bb:c56b with SMTP id z1-20020a05620a260100b006bc70bbc56bmr19178820qko.416.1662025663322;        Thu, 01 Sep 2022 02:47:43 -0700 (PDT)'},
 {'name': 'ARC-Seal',
  'value': 'i=1; a=rsa-sha256; t=1662025663; cv=none;        d=google.com; s=arc-20160816;        b=adeA97WasbFki+m0wp7/zxPoKGH01s5hhDeJyz7C6luEBnu+31t8x2tHcH+bNNBZZq         yV3+blvaNoxvLt7mIVKYCPXpAioaHrvq0+nNBk5lH8awkA7CEaF2fK+WcRRV+4jdE8Kt         Vu3YHI8D3aMvcc7TF1JTk6SZTLYt3RKgaBryq95oSS2KgYk3EupaOlTPViSPFcRjnTSg         CAMG0Hcxk+RVN5i91cV8rIAXzAXVmV/cSm7UenKYbS2eKPFgKLNSszV71kb0W+ov58Za         mp5vZSJCLw2i8f9g6r/Y9puWKNhedP7kvlZE

In [94]:
data

,amount,acct,datetime
0,200.00,162******608,"Tue, 6 Sep 2022 17:25:10 +0530"
1,200.00,162******608,"Mon, 5 Sep 2022 03:34:59 +0530"
2,5100.00,070******357,"Sun, 4 Sep 2022 22:35:43 +0530"
3,5000.00,070******357,"Sun, 4 Sep 2022 22:34:10 +0530"
4,5000.00,162******608,"Sun, 4 Sep 2022 22:34:09 +0530"
5,300.00,162******608,"Sat, 3 Sep 2022 21:30:31 +0530"
6,300.00,070******357,"Fri, 2 Sep 2022 16:04:05 +0530"
7,2100.00,070******357,"Thu, 1 Sep 2022 15:59:39 +0530"
8,5021.50,162******608,"Thu, 1 Sep 2022 15:17:34 +0530"
9,5000.00,070******357,"Thu, 1 Sep 2022 15:17:33 +0530"


In [79]:
trans_lst

[{'trans': 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 200.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name',
  'datetime': 'Tue, 6 Sep 2022 17:25:10 +0530'},
 {'trans': 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 200.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name',
  'datetime': 'Mon, 5 Sep 2022 03:34:59 +0530'},
 {'trans': 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 5100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
  'datetime': 'Sun, 4 Sep 2022 22:35:43 +0530'},
 {'trans': 'Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 5000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
  'datetime': 

In [81]:
transaction = trans_lst[0]['trans']
transaction

'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 200.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name'

In [85]:
pad = re.search('\d*\.+\d+', transaction).group()
print(type(pad))

<class 'str'>


In [53]:
pad = re.findall('\d*\*+\d+', transaction)
print(pad)

['162******608']


In [52]:
pad = re.findall('\d*\.?\d+', transaction)
print(pad[0])

200.00


In [ ]:
id_lst = []
msg_lst = []
for ids in msg:
    id_lst.append(ids['id'])
    
for id in id_lst:
    message = service.users().messages().get(userId='me', id=id).execute()
    # d = ['Debited|Credited']
    pattern = re.compile(r'Debited|Credited')
    snippet = message['snippet']
    if re.search('Debited|Credited', snippet):
        pad = re.findall('\d*\.?\d+', snippet)
        print(pad)
    else:
        pass

['5000.00', '162', '608']
['5000.00', '070', '357']
['10000.00', '162', '608']
['33000.00', '162', '608']


In [27]:
msgs['payload']['headers'][-1]['value']

'Thu, 1 Sep 2022 15:17:33 +0530'

In [ ]:
result = service.users().messages().list(userId='me').execute()
messages = result.get('messages')
msg = messages[:3]
msg

[{'id': '18304d6779c2d13c', 'threadId': '18304d6779c2d13c'},
 {'id': '18304c8e5075a81a', 'threadId': '18304c8e5075a81a'},
 {'id': '1830481aa1352a24', 'threadId': '1830481aa1352a24'}]

In [ ]:
id_lst = []
for ids in msg:
    id_lst.append(ids['id'])
txt = service.users().messages().get(userId='me', id='182fa3a277f6fd04').execute()
txt

{'id': '182fa3a277f6fd04',
 'threadId': '182fa3a277f6fd04',
 'labelIds': ['CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'We noticed a login to your account @GoodrichOkoro from a new device. Was this you? New login Location* Nigeria Device Chrome on Linux *Location is approximate based on the login&#39;s IP address. If',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'dongoodygoody@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:6f02:e:b0:23:4722:82d with SMTP id 14csp288701rcj;        Thu, 1 Sep 2022 11:04:51 -0700 (PDT)'},
   {'name': 'X-Google-Smtp-Source',
    'value': 'AA6agR6gFhT6SvFW3SMW+y7csq7Nz9Ntj8FUFeBjybzm1GjxsvVT9TTVegz9zwo90bpYmfPuCbMl'},
   {'name': 'X-Received',
    'value': 'by 2002:a25:f80c:0:b0:691:7d23:74c9 with SMTP id u12-20020a25f80c000000b006917d2374c9mr21247003ybd.112.1662055491574;        Thu, 01 Sep 2022 11:04:51 -0700 (PDT)'},
   {'name': 'ARC-Seal',
    'value': '

In [ ]:
id_lst = []
msg_lst = []
for ids in msg:
    id_lst.append(ids['id'])
    
for id in id_lst:
    message = service.users().messages().get(userId='me', id=id).execute()
    # d = ['Debited|Credited']
    pattern = re.compile(r'Debited|Credited')
    snippet = message['snippet']
    if re.search('Debited|Credited', snippet):
        msg_lst.append(snippet)
    else:
        pass

print(msg_lst[0])

IndexError: list index out of range

In [ ]:
msg_lst

['Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name',
 'Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 1000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name'

In [ ]:
pat = re.compile(r'\d*\.?\d+')
for x in msg_lst:
   pad = re.findall('\d*\.?\d+', x)

print(pad[0])

11200.00


In [ ]:
you = txt5['snippet']
you

'Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name'

In [ ]:
import re
bank = []
pattern = re.compile(r'Debited')
b = you if re.search('Debited', you) else 3
print(b)

# import re
# pattern=re.compile(r'ack') 
# str_match = [x for x in my_list if re.search('ack', x)]
# print(str_match)

Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name


In [ ]:
ids = [txt1, txt2, txt3, txt4, txt5]
pattern = re.compile(r'Debited|Credited')
for txt in ids:
    snippet = txt['snippet']
    if re.search('Debited|Credited', snippet):
        print(snippet)
    else:
        print('None')

Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
None
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name


In [ ]:
ids = [txt1, txt2, txt3, txt4, txt5]
d = 'Debited|Credited'
pattern = re.compile(r'd')
for txt in ids:
    snippet = txt['snippet']
    if re.search('d', snippet):
        print(snippet)
    else:
        print('None')

Dear GOODRICH IFEANYI OKORO, Your account has been Credited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 162******608 Account Name
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 3100.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name
None
Dear GOODRICH IFEANYI OKORO, Your account has been Debited NGN 2000.00 Kindly click to rate your experience with Access Bank Good Fair Bad Transaction Summary A/C Number 070******357 Account Name


In [ ]:
payload = txt['payload']
headers = payload['headers']

parts = payload.get('parts')[0]
data = parts['body']['data']
data = data.replace('-','+').replace('_','/')
decode_data = base64.b64decode(data)
decode_data

b'Top stories for Goodrich\r\n\r\n-----\r\n\r\nQuestion: How long should it take to learn Python completely and start developing programs?\r\n\r\n\r\nAnswer from Larry Lock\r\n\r\n\r\nI\xe2\x80\x99m a professional programmer (30+ years experience).\r\n\r\nHere are some hints, in addition to an answer to your question:\r\n\r\n1. Most professional programmers never learn a prog \r\nRead More: https://www.quora.com/qemail/tc?al_imp=eyJ0eXBlIjogMzMsICJoYXNoIjogIjE2Nzk1NjAwNDEyOTQwMjczNjd8MXwxfDU1MTYyMjc4In0%3D&al_pri=8&aoid=qopuVW6VbFH&aoty=1&aty=4&cp=1&et=2&id=9f9d2dd44f184eb79b0eee6ba9bdb014&uid=m0UWbQjebdu\r\n-----\r\nQuestion: Why is C++ getting replaced by Rust while C is not?\r\n\r\n\r\nAnswer from Ian Joyner\r\n\r\n\r\nBoth C and C++ should be replaced as system languages. And they should not be used for problem-oriented programming in the first place. Yes, C++ tries to be a language for \r\nRead More: https://www.quora.com/qemail/tc?al_imp=eyJ0eXBlIjogMzMsICJoYXNoIjogIjE2Nzk1NjAwND

In [ ]:
soup = bs(decode_data,"lxml" )
body = soup.body()
body